In [1]:
from google.colab import drive

drive.mount("gdrive")

%cd gdrive/MyDrive/aitalenthack/

Mounted at gdrive
/content/gdrive/MyDrive/aitalenthack


## Load dataset

In [ ]:
def read_dataset(file_name):
    try:
        import pickle

    except:
        !pip install pickle

    with open(file_name, "rb") as handle:
        loaded_file = pickle.load(handle)

    return loaded_file


dataset = read_dataset("unsplash_images/interier_dataset.pickle")
dataset_second_part = read_dataset("unsplash_images/interier_second_part.pickle")

dataset.extend(dataset_second_part)

print(f"Data shape: {len(dataset)}")


Data shape: 2364


In [ ]:
import json

with open("unsplash_images/metadata.jsonl", "w") as file:
    for line in dataset:
        # rename key cause imagefolder works only with certain naming
        line["file_name"] = line.pop("image")
        file.write(json.dumps(line) + '\n')


In [ ]:
try:
    from datasets import load_dataset

except:
    !pip install datasets -U -q
    from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="unsplash_images/", split="train")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


Resolving data files:   0%|          | 0/2367 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.save_to_disk("/content/gdrive/MyDrive/aitalenthack/unsplash_dataset.hf")


Saving the dataset (0/9 shards):   0%|          | 0/2364 [00:00<?, ? examples/s]

## Fine-tune

In [2]:
%cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install -e diffusers/ -q
!ls -la
/

!pip install -r diffusers/examples/text_to_image/requirements_sdxl.txt -q
!pip install accelerate -q
!pip install wandb -q
!pip install -U peft -q
!pip install transformers -q
!pip install xformers -q
!pip install bitsandbytes -q


!accelerate config default


/content
Cloning into 'diffusers'...
remote: Enumerating objects: 63886, done.
remote: Counting objects: 100% (401/401), done.
remote: Compressing objects: 100% (236/236), done.
remote: Total 63886 (delta 243), reused 260 (delta 139), pack-reused 63485
Receiving objects: 100% (63886/63886), 47.54 MiB | 30.73 MiB/s, done.
Resolving deltas: 100% (46731/46731), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for diffusers (pyproject.toml) ... done
total 24
drwxr-xr-x  1 root root 4096 Jun 11 19:32 .
drwxr-xr-x  1 root root 4096 Jun 11 19:30 ..
drwxr-xr-x  4 root root 4096 Jun 10 13:32 .config
drwxr-xr-x 12 root root 4096 Jun 11 19:32 diffusers
drwx------  5 root root 4096 Jun 11 19:32 gdrive
drwxr-xr-x  1 root root 4096 Jun 10 13:38 sample_data
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5

In [3]:
from accelerate.utils import write_basic_config

write_basic_config()


Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


False

In [18]:
!nvidia-smi

Tue Jun 11 20:54:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   65C    P8              18W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [25]:
!accelerate launch diffusers/examples/text_to_image/train_text_to_image_lora_sdxl.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
    --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
    --train_data_dir="/content/gdrive/MyDrive/aitalenthack/unsplash_images" \
    --image_column="image" \
    --caption_column="text" \
    --validation_prompt="living room with a couch, chair, and a table" \
    --num_validation_images=4 \
    --cache_dir="/content/cache" \
    --gradient_accumulation_steps=1 --gradient_checkpointing \
    --use_8bit_adam \
    --resolution=1024 \
    --random_flip \
    --center_crop \
    --enable_xformers_memory_efficient_attention \
    --train_batch_size=8 \
    --max_train_steps=10000 \
    --checkpointing_steps=500 \
    --checkpoints_total_limit=2 \
    --mixed_precision="fp16" \
    --learning_rate=1e-04 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --seed=42 \
    --output_dir="/content/gdrive/MyDrive/aitalenthack/sdxl_lora_weights" \
    --report_to="wandb"


/content/diffusers/src/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
2024-06-11 21:29:15.385463: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 21:29:15.452538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 21:29:15.452601